In [1]:
versioninfo()

Julia Version 1.5.1
Commit 697e782ab8 (2020-08-25 20:08 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


In [ ]:
]activate .

In [2]:
]instantiate

In [3]:
using Squareness

## Simple Newton Method

In [4]:
function _issquare_with_simple_newton(x::I) where {I <: Integer}
    x == 0 && return true
    xtz = trailing_zeros(x)
    isodd(xtz) && return false
    x >>= unsigned(xtz)
    o_xrt::I = x
    xrt::I = (x + 0x1) >> 0x1
    xrt == 0 && return false  # overflow
    while xrt < o_xrt
        o_xrt = xrt
        xrt = (xrt + x ÷ xrt) >> 0x1
    end
    xrt * xrt == x
end

issquare_with_simple_newton(x::Signed) = x < 0 ? false : _issquare_with_simple_newton(x % Unsigned)
issquare_with_simple_newton(x::Unsigned) = _issquare_with_simple_newton(x)
issquare_with_simple_newton(x::BigInt) = x < 0 ? false : _issquare_with_simple_newton(x)

issquare_with_simple_newton (generic function with 3 methods)

### Simple Check

In [5]:
all(issquare(x) == issquare_with_simple_newton(x) == Squareness.issquare_with_isqrt(x) for x in rand(Int, 10000))

true

In [6]:
all(issquare(x) == issquare_with_simple_newton(x) == Squareness.issquare_with_isqrt(x) for x in rand(Int32, 10000))

true

## Benchmark

In [7]:
using BenchmarkTools

### Compare `issquare()`(with P-adic Newton) vs `issquare_with_isqrt()` vs `issquare_with_simple_newton()`

#### `Int64`

In [8]:
rand_int64() = rand(Int64) & typemax(Int64)

rand_int64 (generic function with 1 method)

In [9]:
@benchmark issquare(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.946 ns (0.00% GC)
  median time:      5.248 ns (0.00% GC)
  mean time:        4.737 ns (0.00% GC)
  maximum time:     38.599 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [10]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.297 ns (0.00% GC)
  median time:      7.327 ns (0.00% GC)
  mean time:        7.690 ns (0.00% GC)
  maximum time:     53.945 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [11]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.346 ns (0.00% GC)
  median time:      318.233 ns (0.00% GC)
  mean time:        223.376 ns (0.00% GC)
  maximum time:     614.047 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int64` × 100

In [12]:
rand_int64(N) = rand(Int64, N) .& typemax(Int64)

rand_int64 (generic function with 2 methods)

In [13]:
@benchmark issquare.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     474.588 ns (0.00% GC)
  median time:      569.273 ns (0.00% GC)
  mean time:        589.108 ns (1.25% GC)
  maximum time:     15.586 μs (96.05% GC)
  --------------
  samples:          10000
  evals/sample:     194

In [14]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     338.957 ns (0.00% GC)
  median time:      362.410 ns (0.00% GC)
  mean time:        379.563 ns (2.21% GC)
  maximum time:     14.676 μs (97.32% GC)
  --------------
  samples:          10000
  evals/sample:     210

In [15]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     16.207 μs (0.00% GC)
  median time:      22.601 μs (0.00% GC)
  mean time:        23.060 μs (0.00% GC)
  maximum time:     79.982 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

#### `Int32`

In [16]:
rand_int32() = rand(Int32) & typemax(Int32)

rand_int32 (generic function with 1 method)

In [17]:
@benchmark issquare(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.880 ns (0.00% GC)
  median time:      4.322 ns (0.00% GC)
  mean time:        4.146 ns (0.00% GC)
  maximum time:     33.974 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [18]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.428 ns (0.00% GC)
  median time:      7.456 ns (0.00% GC)
  mean time:        7.549 ns (0.00% GC)
  maximum time:     44.383 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [19]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.585 ns (0.00% GC)
  median time:      98.625 ns (0.00% GC)
  mean time:        69.919 ns (0.00% GC)
  maximum time:     259.276 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int32` × 100

In [20]:
rand_int32(N) = rand(Int32, N) .& typemax(Int32)

rand_int32 (generic function with 2 methods)

In [21]:
@benchmark issquare.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     417.176 ns (0.00% GC)
  median time:      504.842 ns (0.00% GC)
  mean time:        523.611 ns (1.45% GC)
  maximum time:     16.035 μs (96.71% GC)
  --------------
  samples:          10000
  evals/sample:     199

In [22]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     366.965 ns (0.00% GC)
  median time:      397.040 ns (0.00% GC)
  mean time:        413.289 ns (1.88% GC)
  maximum time:     16.385 μs (97.48% GC)
  --------------
  samples:          10000
  evals/sample:     201

In [23]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     5.539 μs (0.00% GC)
  median time:      7.767 μs (0.00% GC)
  mean time:        7.889 μs (0.00% GC)
  maximum time:     21.532 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     6

#### `Int16`

In [24]:
rand_int16() = rand(Int16) & typemax(Int16)

rand_int16 (generic function with 1 method)

In [25]:
@benchmark issquare(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.647 ns (0.00% GC)
  median time:      5.524 ns (0.00% GC)
  mean time:        4.765 ns (0.00% GC)
  maximum time:     35.386 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [26]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.229 ns (0.00% GC)
  median time:      7.417 ns (0.00% GC)
  mean time:        7.480 ns (0.00% GC)
  maximum time:     39.636 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [27]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.374 ns (0.00% GC)
  median time:      51.880 ns (0.00% GC)
  mean time:        38.604 ns (0.00% GC)
  maximum time:     145.601 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int16` × 100

In [28]:
rand_int16(N) = rand(Int16, N) .& typemax(Int16)

rand_int16 (generic function with 2 methods)

In [29]:
@benchmark issquare.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     364.132 ns (0.00% GC)
  median time:      426.475 ns (0.00% GC)
  mean time:        445.122 ns (1.86% GC)
  maximum time:     20.976 μs (97.69% GC)
  --------------
  samples:          10000
  evals/sample:     204

In [30]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     382.285 ns (0.00% GC)
  median time:      405.098 ns (0.00% GC)
  mean time:        425.499 ns (1.91% GC)
  maximum time:     17.447 μs (96.69% GC)
  --------------
  samples:          10000
  evals/sample:     200

In [31]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     2.418 μs (0.00% GC)
  median time:      3.325 μs (0.00% GC)
  mean time:        3.401 μs (0.00% GC)
  maximum time:     12.587 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9

#### `Int128`

In [32]:
rand_int128() = rand(Int128) & typemax(Int128)

rand_int128 (generic function with 1 method)

In [33]:
@benchmark issquare(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.119 ns (0.00% GC)
  median time:      24.371 ns (0.00% GC)
  mean time:        17.634 ns (0.00% GC)
  maximum time:     70.616 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [34]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.123 ns (0.00% GC)
  median time:      12.697 ns (0.00% GC)
  mean time:        12.900 ns (0.00% GC)
  maximum time:     50.152 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [35]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.046 ns (0.00% GC)
  median time:      1.794 μs (0.00% GC)
  mean time:        1.237 μs (0.00% GC)
  maximum time:     2.626 μs (0.00% GC)
  --------------
  samples:          4042
  evals/sample:     1000

#### `Int128` × 100

In [36]:
rand_int128(N) = rand(Int128, N) .& typemax(Int128)

rand_int128 (generic function with 2 methods)

In [37]:
@benchmark issquare.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.477 μs (0.00% GC)
  median time:      1.916 μs (0.00% GC)
  mean time:        1.981 μs (0.89% GC)
  maximum time:     177.917 μs (98.68% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [38]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.438 μs (0.00% GC)
  median time:      1.513 μs (0.00% GC)
  mean time:        1.543 μs (1.12% GC)
  maximum time:     173.869 μs (99.02% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [39]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     96.740 μs (0.00% GC)
  median time:      127.650 μs (0.00% GC)
  mean time:        129.819 μs (0.00% GC)
  maximum time:     340.463 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

#### `BigInt`

In [40]:
rand_mpz() = (big(rand(UInt128)) << rand(0:128)) + rand(UInt128)

rand_mpz (generic function with 1 method)

In [41]:
# @benchmark issquare(n) setup=(n=rand_mpz())
@benchmark Squareness.issquare_with_padic_newton(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.201 ns (0.00% GC)
  median time:      299.549 μs (1.29% GC)
  mean time:        314.849 μs (2.93% GC)
  maximum time:     703.159 μs (14.58% GC)
  --------------
  samples:          17
  evals/sample:     999

In [42]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  88 bytes
  allocs estimate:  5
  --------------
  minimum time:     135.587 ns (0.00% GC)
  median time:      243.179 ns (0.00% GC)
  mean time:        392.677 ns (20.70% GC)
  maximum time:     180.977 μs (75.75% GC)
  --------------
  samples:          10000
  evals/sample:     550

In [43]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.182 ns (0.00% GC)
  median time:      18.831 μs (0.00% GC)
  mean time:        27.694 μs (24.22% GC)
  maximum time:     132.676 μs (56.30% GC)
  --------------
  samples:          181
  evals/sample:     999